# Quantum Teleportation in Cirq (Basic Version)

This notebook walks through constructing a minimal quantum teleportation protocol using raw Cirq operations (no custom gates). We'll explore entanglement, state preparation, measurement, and classical feed-forward corrections.

## 1. Imports

In [ ]:
import cirq
from typing import Sequence

## 2. Helper Functions
We define small helpers for Bell pair preparation and message encoding.

In [ ]:
def PrepareBellPair(q1: cirq.Qid, q2: cirq.Qid) -> Sequence[cirq.Operation]:
    return [
        cirq.H.on(q1),
        cirq.CNOT.on(q1, q2)
    ]

def PrepareMessage(alice: cirq.Qid, message: cirq.Qid) -> Sequence[cirq.Operation]:
    return [
        cirq.CNOT.on(message, alice),
        cirq.H.on(message)
    ]

## 3. Building the Teleportation Operation List
We build the protocol step-by-step.

In [ ]:
def simple_teleportation() -> Sequence[cirq.Operation]:
    alice = cirq.NamedQubit('alice')
    bob = cirq.NamedQubit('bob')
    program: Sequence[cirq.Operation] = []
    # 1. Create entangled pair shared by Alice and Bob
    program.extend(PrepareBellPair(alice, bob))
    # 2. Prepare a message qubit (state to teleport)
    msg = cirq.NamedQubit('msg')
    program.append(cirq.rx(0.7)(msg))  # Example arbitrary state
    # 3. Entangle message with Alice's half
    program.extend(PrepareMessage(alice, msg))
    # 4. Measure Alice + msg to obtain classical bits
    program.append(cirq.measure(msg, key='b1'))
    program.append(cirq.measure(alice, key='b2'))
    # 5. Classical feed-forward corrections on Bob
    program.append(cirq.Z.on(bob).with_classical_controls('b1'))
    program.append(cirq.X.on(bob).with_classical_controls('b2'))
    # 6. Measure Bob's qubit directly to sample teleported state
    program.append(cirq.measure(bob, key='result'))
    return program

## 4. Circuit Construction and Display

In [ ]:
program = simple_teleportation()
circuit = cirq.Circuit(program)
print(circuit)

## 5. Simulation
We simulate many repetitions to get statistics of the final correction correctness.

In [ ]:
import math

sim = cirq.Simulator()
result = sim.run(cirq.Circuit(simple_teleportation()), repetitions=1000)
hist = result.histogram(key='result')
print("Histogram of Bob's measurements (teleported state):")
print(hist)
# Expected probabilities for |ψ> = Rx(0.7)|0>
p0 = math.cos(0.35)**2
p1 = math.sin(0.35)**2
print(f"Expected P(0)≈{p0:.3f}, P(1)≈{p1:.3f}")

## 6. Interpreting Results
If teleportation succeeds, Bob's qubit should be in the same state |ψ> that was originally prepared on the message qubit. Since we prepared |ψ> = Rx(0.7)|0>, the measurement statistics should reflect the amplitudes of that state: P(0) ≈ cos^2(0.35) ≈ 0.88 and P(1) ≈ sin^2(0.35) ≈ 0.12.

## 7. Next Steps
Try varying the rotation angle, or replacing `rx(0.7)` with other single-qubit gates (e.g. `cirq.ry(θ)`). Compare outcomes with and without classical control corrections.